In [2]:
#Import Libraries
import requests as rq
from bs4 import BeautifulSoup
import html5lib
import lxml
import selenium
import time
from selenium import webdriver
import pandas as pd

<font size=4>Use Selenium to scrape data from the web</font>

In [3]:
#Function to get every date from a range
def Print_every_day(start,end):
    date=[]

    month31=[1,3,5,7,8,10,12]
    month30=[4,6,9,11]

    year=int(start[:4])
    month=int(start[4:6])
    day=int(start[6:8])
    
    year_end=int(end[:4])
    moth_end=int(end[4:6])
    day_end=int(end[6:8])

    while (year<year_end) or (year==year_end and month<moth_end) or (year==year_end and month==moth_end and day<day_end):


        if year%4==0:#Check if it is a leap year

            if month<12:#Check if I'm at the end of the month

                if (month in month31): 
                    if day<31:
                        day+=1
                    else:
                        month+=1
                        day=1

                elif (month in month30): 
                    if day<30:
                        day+=1
                    else:
                        month+=1
                        day=1

                elif day<29: #February
                    day+=1
                else:
                    month+=1
                    day=1
            else:
                if day<31:
                    day+=1
                else:
                    year+=1
                    month=1
                    day=1
        else:
            if month<12:

                if (month in month31): 
                    if day<31:
                        day+=1
                    else:
                        month+=1
                        day=1

                elif (month in month30): 
                    if day<30:
                        day+=1
                    else:
                        month+=1
                        day=1

                elif day<28: 
                    day+=1
                else:
                    month+=1
                    day=1
            else:
                if day<31:
                    day+=1
                else:
                    year+=1
                    month=1
                    day=1


        if day<10: #Check if I need to add a 0 in front of the number
            day=str(day)
            day="0"+day
        else:
            day=str(day)

        if month<10: #Check if I need to add a 0 in front of the month
            month=str(month)
            month="0"+month
        else:
            month=str(month)
        
        year=str(year)

        date.append(year+month+day) #Add my date to the date list
        #Reconvert in int so that i can do operation on them
        year=int(year)
        month=int(month)
        day=int(day)
    
    return date

<font size=5>Final Code</font>

In [4]:

inizio="20160719" #I have to start one day earlier to the one i want to actual start from
fine="20160720"

sleep_for=1 
sleep=1
Date=Print_every_day(inizio,fine) 

#Cycle every page untill i get to the end
for j in range(len(Date)):
    
    url=f"https://coinmarketcap.com/historical/{Date[j]}/"
    driver=webdriver.Chrome()
    driver.get(url)
    time.sleep(sleep_for)
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    screen_height = driver.execute_script("return window.screen.height;") 

    
    i=1
    while True:
        
        step=screen_height * i
        driver.execute_script(f"window.scrollTo(0, {step});")
        i += 1
        time.sleep(sleep)

        
        if step> scroll_height:
            break

    #Extract data from the html
    html = driver.page_source 
    driver.quit()
    soup=BeautifulSoup(html,"html.parser")
    righe=soup.find_all("tr",class_="cmc-table-row")#Load all row of the table


    Rank=[]
    Ticket=[]
    Market_Cap=[]
    Vol=[]
    Price=[]
    Supply=[]
    Change1h=[]
    Change24h=[]
    Change7d=[]
    Historical_Data_1w=pd.DataFrame()



    #Take all data that i need
    for x in righe:
        num=x.find("td",class_="cmc-table__cell cmc-table__cell--sticky cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__rank")
        Symbol=x.find("td",class_="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--left cmc-table__cell--sort-by__symbol")
        MC=x.find("td",class_="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__market-cap")
        Volume=x.find("td",class_="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__volume-24-h")
        PriceData=x.find("td",class_="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__price")
        Circulating_Supply=x.find("td",class_="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__circulating-supply")
        Change_1h=x.find("td",class_="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__percent-change-1-h")
        Change_24h=x.find("td",class_="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__percent-change-24-h")
        Change_7d=x.find("td",class_="cmc-table__cell cmc-table__cell--sortable cmc-table__cell--right cmc-table__cell--sort-by__percent-change-7-d")

        num=num.text
        if "--" in num:
            num=num.replace("--","")
            Rank.append(int(num))
        else:
            Rank.append(int(num))
        
        Symbol=Symbol.text
        if "--" in Symbol:
            Symbol=Symbol.replace("--","")
            Ticket.append(Symbol)
        else:
            Ticket.append(Symbol)

        MC=MC.text
        MC = MC.replace(',', '')
        MC = MC.replace('$', '')
        if "--" in MC:
            MC=MC.replace("--","")
            Market_Cap.append(MC)
        else:
            Market_Cap.append(float(MC))
        
        if Volume!=None:
            Volume=Volume.text
            Volume = Volume.replace(',', '')
            Volume = Volume.replace('$', '')
            if "<" in Volume:
                Volume = Volume.replace('<', '')
            if ">" in Volume:
                Volume = Volume.replace('>', '')
            if "--" in Volume:
                Volume=Volume.replace("--","")
                Vol.append(Volume)
            else:
                Vol.append(float(Volume))
        else:
            Vol.append(None)

        PriceData=PriceData.text
        PriceData = PriceData.replace(',', '')
        PriceData = PriceData.replace('$', '')
        if "--" in PriceData:
            PriceData=PriceData.replace("--","")
            Price.append(PriceData)
        else:
            Price.append(float(PriceData))

        Circulating_Supply=Circulating_Supply.text
        Circulating_Supply = Circulating_Supply.replace('.', '')
        
        Circulating_Supply = Circulating_Supply.replace('*', '')
        Circulating_Supply = Circulating_Supply.replace(Symbol, '')
        if "--" in Circulating_Supply:
            Circulating_Supply=Circulating_Supply.replace("--","")
            Supply.append(Circulating_Supply)
        else:
            Supply.append(float(Circulating_Supply))


        Change_1h=Change_1h.text
        Change_1h = Change_1h.replace(',', '.')
        Change_1h = Change_1h.replace('%', '')
        if "<" in Change_1h:
            Change_1h = Change_1h.replace('<', '')
        if ">" in Change_1h:
            Change_1h = Change_1h.replace('>', '')
        if "--" in Change_1h:
            Change_1h=Change_1h.replace("--","")
            Change1h.append(Change_1h)
        else:
            if len(Change_1h)>=8:#To account for changes 1.046,36% so that i can format them
                Change_1h=Change_1h.replace(".","")
                Change_1h=(float(Change_1h))/100 #Now it would be something like 104636, so i divide by 100 to get the original value
                Change1h.append(Change_1h)
            else:
                Change1h.append(float(Change_1h))

        Change_24h=Change_24h.text
        Change_24h = Change_24h.replace(',', '.')
        Change_24h = Change_24h.replace('%', "")
        if "<" in Change_24h:
            Change_24h = Change_24h.replace('<', '')
        if ">" in Change_24h:
            Change_24h = Change_24h.replace('>', '')
        if "--" in Change_24h:
            Change_24h = Change_24h.replace('--', "")
            Change24h.append(Change_24h)
        else:
            if len(Change_24h)>=8:
                Change_24h=Change_24h.replace(".","")
                Change_24h=(float(Change_24h))/100
                Change24h.append(Change_24h)
            else:
                Change24h.append(float(Change_24h))
  
        Change_7d=Change_7d.text
        Change_7d = Change_7d.replace(',', '.')
        Change_7d = Change_7d.replace('%', '')
        if "<" in Change_7d:
            Change_7d = Change_7d.replace('<', "")
        if ">" in Change_7d:
            Change_7d = Change_7d.replace('>', '')
        if "--" in Change_7d:
            Change_7d = Change_7d.replace('--', "")
            Change7d.append(Change_7d)
        else:
            if len(Change_7d)>=8:
                Change_7d=Change_7d.replace(".","")
                Change_7d=(float(Change_7d))/100 
                Change7d.append(Change_7d)
            else:
                Change7d.append(float(Change_7d))
            
        #Put everything inside a dataframe, and then inside a csv
        Historical_Data_1w=pd.DataFrame({"Rank":Rank,"Symbol":Ticket,"Market Cap":Market_Cap,"Volume":Vol,"Prezzo":Price,  
                                 "Circulating":Supply,"1h Variation":Change1h,"24h Variation":Change24h,"7d Variation":Change7d})
        Historical_Data_1w.to_csv(f"Hystorical Data CoinMarketCap/{Date[j]}.csv") 
        
        
        

        

<font size=5>Take the list of the Stable Coin and Asset Backed Crypto, so that i can remove them when i calculate the ALT season index</font>

In [ ]:
#Take the ticket of the stable Coin and asset backed tocken
urls=["https://coinmarketcap.com/view/stablecoin/","https://www.coingecko.com/en/categories/asset-backed-tokens"]

#Cycle every page untill the end date
url="https://coinmarketcap.com/view/stablecoin/"
Information=rq.get(url)



soup=BeautifulSoup(Information.content,"html.parser")

#Load all row of the table
r1=soup.find_all("tr",style="cursor:pointer")
r2=soup.find_all("tr",class_="sc-240ce903-0 iPeVVh")

Ticket=[]


for x in r1:
    Symbol=x.find("div",class_="sc-1c5f2868-3 hHWqgz")
    
    Ticket.append(Symbol.text)

for x in r2:
    
    
    Symbol=x.find("span",class_="crypto-symbol") #the class_ might change 
    Ticket.append(Symbol.text)

Stable_Coin=pd.Series(Ticket)
Stable_Coin.to_csv("Stable Coin and Asset Backed Token/Stable Coin List.csv")

In [ ]:
#Take the Ticket of the asset backed token
url="https://www.coingecko.com/en/categories/asset-backed-tokens"

driver=webdriver.Chrome()
driver.get(url)

scroll_height = driver.execute_script("return document.body.scrollHeight;")
screen_height = driver.execute_script("return window.screen.height;") 


i=1
while True:
    
    step=screen_height * i
    driver.execute_script(f"window.scrollTo(0, {step});")
    i += 1
    time.sleep(sleep)

    
    if step> scroll_height:
        break

    

html = driver.page_source 
driver.quit()


soup=BeautifulSoup(html,"html.parser")


r3=soup.find_all("tr",class_="hover:tw-bg-gray-50 tw-bg-white dark:tw-bg-moon-900 hover:dark:tw-bg-moon-800 tw-text-sm")




In [133]:
Ticket=[]

#Take every data i need from each row of the table
for x in r3:
    Symbol=x.find("div",class_="tw-block 2lg:tw-inline tw-text-xs tw-leading-4 tw-text-gray-500 dark:tw-text-moon-200 tw-font-medium")
    Symbol=Symbol.text

    MarketCap=x.find("td",class_="tw-text-end tw-px-1 tw-py-2.5 2lg:tw-p-2.5 tw-bg-inherit tw-text-gray-900 dark:tw-text-moon-50")
    

    Final_Symbol=""
    for ln in range(len(Symbol)):
        
        dummy=""

        if ln>2 and ln<range(len(Symbol))[-1] and Symbol[ln]!="":
            dummy = Symbol[ln]

        Final_Symbol+=dummy
    
    Ticket.append(Final_Symbol)

Ticket=Ticket[:32] #Take only the first 32, since the rest dont even have a Market Cap
print(Ticket)


Assets_Backed_Token=pd.Series(Ticket)
Assets_Backed_Token.to_csv("Crypto Type List/Asset Backed Token List.csv")

['XAUT', 'PAXG', 'CETH', 'CWBTC', 'TBTC', 'OHM', 'CUSDC', 'CDAI', 'SB', 'SETH', 'TEMPLE', 'CBAT', 'SUSD', 'RENBTC', 'FEI', 'CUNI', 'SBTC', 'TXAU', 'RAI', 'SHOPX', 'VEUR', 'CZRX', 'CLINK', 'TXAG', 'SEUR', 'XAUR', 'AX', 'VNXAU', 'DGX', 'ZUSD', 'TIME', 'PBTC']
